In [7]:
from echr import *
from classifier import *
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
from datasets import DatasetDict
from scipy.special import expit as sigmoid  
import json
import time  
from transformers import EarlyStoppingCallback
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [8]:
all_datasets = {
    'test': {
        'chamber': pd.read_csv('datasets/test_chamber.csv'),
        'grand_chamber': pd.read_csv('datasets/test_grand_chamber.csv'),
    },
    'train': {
        'chamber': {idx: pd.read_csv('datasets/train_chamber_'+str(idx)+'.csv') for idx in range(0,7)},    
        'grand_chamber': {idx: pd.read_csv('datasets/train_grand_chamber_'+str(idx)+'.csv') for idx in range(0,7)},    
    }
}

In [9]:
all_datasets['train']['grand_chamber'][0].head(5)

,unique_id,id,body,text,year,violation,sample_weight,judgment_info
0,147,001-58593,Grand Chamber,7.The applicant was born in August 1982. On 12...,1999,1,1.0,"FOR THESE REASONS, THE COURT\n1.\tDISMISSES UN..."
1,8,001-57980,Grand Chamber,11. The applicant was arrested by police offic...,1996,0,1.0,"FOR THESE REASONS, THE COURT\n1. HOLDS BY FO..."
2,120,001-59454,Grand Chamber,13.The complaints raised in this application a...,2001,0,1.0,"FOR THESE REASONS, THE COURT REJECTS THE APPLI..."
3,111,001-61075,Grand Chamber,11.The applicant was born in 1940 and lives in...,2003,0,1.0,"FOR THESE REASONS, THE COURT\n1. HOLDS UNANIM..."
4,26,001-58847,Grand Chamber,"9.The applicant, who was born in 1958 in Tunis...",2000,0,1.0,"FOR THESE REASONS, THE COURT\nHOLDS BY FIFTEEN..."


In [10]:
build_training_set(all_datasets, 0, method='mexpall').head(5)

,unique_id,id,body,text,year,violation,sample_weight,judgment_info,votes_for
0,682,001-101173,Chamber,5.The applicant was born in 1973 and is curren...,2010,0,1.0,"FOR THESE REASONS, THE COURT UNANIMOUSLY\n1. ...",0.0
1,2007,001-60424,Chamber,8.The applicants teach as temporary staff at t...,2002,1,1.0,"FOR THESE REASONS, THE COURT UNANIMOUSLY\n1. ...",7.0
2,354,001-105353,Chamber,6.The applicants were born in 1957 and 1986 re...,2011,0,1.0,"FOR THESE REASONS, THE COURT\n1. DECLARES UNA...",0.0
3,5918,001-95191,Chamber,5.The applicant was born in 1961 and lives in ...,2009,1,1.0,"FOR THESE REASONS, THE COURT UNANIMOUSLY\n1. ...",7.0
4,4967,001-58102,Chamber,6.The facts of the case as established by the ...,1997,0,1.0,"FOR THESE REASONS, THE COURT\n1.\tDISMISSES UN...",0.0


#### Use longformer

In [6]:
# Parameters
num_splits = len(all_datasets['train']['chamber'])
methods =  ['wcorr', 'obs', 'obs_ip', 'nn', 'mexpall', 'mexpmax', 'mexpmin', 'mexpagr', 'mexpavg']
# methods =  ['wcorr']
early_stopping_patience = 3
run = 'epochs_3_batch_2/'
output_dir = 'results/' + run 
os.makedirs(output_dir, exist_ok=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./longformer_grandchamber',
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,   
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    fp16=True,
    report_to='none' 
)

In [7]:
# # Tokenizer and model
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokenize_fn = partial(preprocess_and_tokenize, tokenizer=tokenizer)
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Test sets
c_test_dataset = prepare_dataset(all_datasets['test']['chamber'])
gc_test_dataset = prepare_dataset(all_datasets['test']['grand_chamber'])

# Tokenize
c_test_dataset = c_test_dataset.map(tokenize_fn, batched=True, remove_columns=["text"])
gc_test_dataset = gc_test_dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

#Rename labels
c_test_dataset = c_test_dataset.rename_column('violation', 'labels')
gc_test_dataset = gc_test_dataset.rename_column('violation', 'labels')

# Format incl labels
c_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
gc_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/871 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

In [9]:
# Run experiment
for split_id in range(num_splits):
    for method in methods:
        if 'obs' not in method: continue
        if split_id == 0: continue

        # Increase num epochs for obs as it has less instances
        og_epochs = training_args.num_train_epochs
        if 'obs' in method:
            training_args.num_train_epochs *= 10
        start_time = time.time()

        print(f"\n🧪 {'='*50}")
        print(f"🔁 Running experiment | Split: {split_id} | Method: {method}")
        print(f"{'='*55}")

        # Prepare folders to store results
        model_path = 'results/'+run+'models/' + method + '_' + str(split_id) + '/'
        os.makedirs(model_path, exist_ok=True)
        training_args.output_dir = model_path

        # Prepare raw dataset and split off 15% for validation
        raw_dataset = build_training_set(all_datasets, split_id, method=method)
        train_df, val_df = train_test_split(raw_dataset, test_size=0.15, stratify=raw_dataset['violation'], random_state=42)
        
        # Convert to HF Datasets
        raw_train = prepare_dataset(train_df)
        raw_val = prepare_dataset(val_df)
        print(f"📊 Dataset sizes → Train: {len(raw_train)}, Val: {len(raw_val)}")

        # Tokenize train and val 
        train_dataset = raw_train.map(tokenize_fn, batched=True, remove_columns=["text"])
        train_dataset = train_dataset.rename_column('violation', 'labels')
        train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
        
        val_dataset = raw_val.map(tokenize_fn, batched=True, remove_columns=["text"])
        val_dataset = val_dataset.rename_column('violation', 'labels')
        val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
        print(f"✅ Val and train dataset tokenized and formatted.")


        # Initialize trainer with weights
        trainer = WeightedTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            method=method,
        )
        
        print("🚀 Starting training...")
        trainer.train()
        print("\n✨ Training complete.")

        print(f"🔍 Evaluating on Chamber test set ({len(all_datasets['test']['chamber'])})...")
        c_preds = trainer.predict(c_test_dataset)
        print_metrics("🏛 Chamber Test Set", c_preds)
        
        print(f"🔍 Evaluating on Grand Chamber test set ({len(all_datasets['test']['grand_chamber'])})...")
        gc_preds = trainer.predict(gc_test_dataset)
        print_metrics("⚖️ Grand Chamber Test Set", gc_preds)
        
        # Extract probabilities for class 1 using sigmoid
        c_pred_probs = get_class1_probs(c_preds.predictions)
        gc_pred_probs = get_class1_probs(gc_preds.predictions)
        
        # Prepare results dict to save
        results = {
            "split_id": split_id,
            "method": method,
            "chamber_test_metrics": c_preds.metrics,
            "grand_chamber_test_metrics": gc_preds.metrics,
            "chamber_test_predictions": {
                "predictions": c_pred_probs.tolist(),
                "labels": c_preds.label_ids.tolist()
            },
            "grand_chamber_test_predictions": {
                "predictions": gc_pred_probs.tolist(),
                "labels": gc_preds.label_ids.tolist()
            }
        }

        # Save results as JSON per method per split
        filename = f"results_split_{split_id}_method_{method}.json"
        filepath = os.path.join(output_dir, filename)
        with open(filepath, 'w') as f:
            json.dump(results, f, indent=4)

        elapsed = time.time() - start_time
        minutes, seconds = divmod(elapsed, 60)
        print(f"⏱️ Duration for split {split_id} | method {method}: {int(minutes)}m {seconds:.2f}s")
        print(f"💾 Saved results to {filepath}")
        training_args.num_train_epochs = og_epochs

print("🎉 All experiments completed successfully and results saved to disk.")


🧪 ==================================================
🔁 Running experiment | Split: 1 | Method: obs
📊 Dataset sizes → Train: 98, Val: 18


Map:   0%|          | 0/98 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

✅ Val and train dataset tokenized and formatted.
🚀 Starting training...


Initializing global attention on CLS token...


...

✅ Val and train dataset tokenized and formatted.
🚀 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Mcc
1,0.384800,0.002729,1.000000,1.000000,1.000000
2,0.438000,0.003500,1.000000,1.000000,1.000000
3,0.187800,0.004277,1.000000,1.000000,1.000000
4,0.090800,0.003303,1.000000,1.000000,1.000000
5,0.051100,0.002961,1.000000,1.000000,1.000000
6,0.071700,0.002309,1.000000,1.000000,1.000000
7,0.082900,0.002081,1.000000,1.000000,1.000000
8,0.059600,0.002531,1.000000,1.000000,1.000000
9,0.075800,0.001619,1.000000,1.000000,1.000000
10,0.076000,0.001677,1.000000,1.000000,1.000000



✨ Training complete.
🔍 Evaluating on Chamber test set (871)...



📊 🏛 Chamber Test Set Results:
  test_loss           : 2.9376
  test_accuracy       : 0.5591
  test_f1             : 0.6718
  test_mcc            : 0.0036
  test_runtime        : 138.4610
  test_samples_per_second: 6.2910
  test_steps_per_second: 3.1490
🔍 Evaluating on Grand Chamber test set (34)...



📊 ⚖️ Grand Chamber Test Set Results:
  test_loss           : 2.1913
  test_accuracy       : 0.6471
  test_f1             : 0.7143
  test_mcc            : 0.3841
  test_runtime        : 5.4202
  test_samples_per_second: 6.2730
  test_steps_per_second: 3.1360
⏱️ Duration for split 6 | method obs_ip: 18m 46.96s
💾 Saved results to results/epochs_3_batch_2/results_split_6_method_obs_ip.json
🎉 All experiments completed successfully and results saved to disk.
